<a href="https://colab.research.google.com/github/inyminymoe/Data-Analysis-with-Open-Source/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4%EA%B8%B0%EB%B0%98%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D(%EC%A4%91%EA%B0%84%EA%B3%BC%EC%A0%9C%EB%AC%BC)_%EA%B0%95%EC%9D%B8%EC%98%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

API 호출

**1: API 호출 및 데이터 수집**

In [20]:
# 설치 및 import
!pip install pandas requests

import requests
import pandas as pd
import matplotlib.pyplot as plt

In [17]:
# 데이터 파악
df_raw = pd.DataFrame(all_data)
print("전체 행수:", len(df_raw))
display(df_raw.head())

print("=== 실제 컬럼 목록 ===")
print(sorted(df_raw.columns.tolist()))

for col in ["MM_TYPE"]:
    if col in df_raw.columns:
        print(f"\n=== {col} 유니크 값(상위 30개) ===")
        vc = df_raw[col].value_counts(dropna=False)
        display(vc.head(30))
    else:
        print(f"\n=== {col} 컬럼 없음 ===")


df = df_raw.copy()

# YEAR/MON 숫자화
df["year"]  = pd.to_numeric(df["YEAR"], errors="coerce").astype("Int64")
df["month"] = pd.to_numeric(df["MON"],  errors="coerce").astype("Int64")

# 개인만 추출
df_person = df[df["MM_TYPE"] == "개인"].copy()
print("개인 행 수:", len(df_person))
display(df_person.head())

전체 행수: 42148


,YEAR,MON,HOSTCODE,DONGCODE,MM_TYPE,TYPE_DIV,TYPE_DIV_CD,CNT,EUS,EUS1,...,NOW_CO2,YEAR_CO2,AVG_CO2,TOTAL_CO2_YEAR,TOTAL_CO2_AVG,TOTAL_RATE_YEAR,TOTAL_RATE_AVG,REG_DATE,year,month
0,2015,01,중랑구,면목3.8동,개인,아파트,TB_APT_C,1,215,250,...,91.16,106,106,-14.84,-14.84,-14,-14,2015-06-04 17:03:55.0,2015,1
1,2015,01,강동구,강일동,개인,아파트,TB_APT_Y,4969,1369580,1398337,...,678764.892,688765.44,693140.29,-10000.548,-14375.398,-1.452,-2.074,2015-06-04 17:03:55.0,2015,1
2,2015,01,강동구,강일동,개인,아파트외,TB_APT_N,14,3639,3014,...,3338.2952,2886.348,3339.912,451.9472,-1.6168,15.658,-0.048,2015-06-04 17:03:55.0,2015,1
3,2015,01,강동구,상일1동,개인,아파트,TB_APT_Y,2205,376695,377894,...,459449.096,446388.6,474691.028,13060.496,-15241.932,2.926,-3.211,2015-06-04 17:03:55.0,2015,1
4,2015,01,강동구,상일1동,개인,아파트외,TB_APT_N,1393,173914,172003,...,464982.06,444897.4824,484627.4204,20084.5776,-19645.3604,4.514,-4.054,2015-06-04 17:03:55.0,2015,1


=== 실제 컬럼 목록 ===
['AVG_CO2', 'CNT', 'DONGCODE', 'ECO2_1', 'ECO2_1_MATH', 'ECO2_1_MATH_YEAR', 'ECO2_2', 'ECO2_2_MATH', 'ECO2_2_MATH_YEAR', 'EUS', 'EUS1', 'EUS2', 'E_RATE_AVG', 'E_RATE_YEAR', 'E_TOE_AVG', 'E_TOE_YEAR', 'GCO2_1', 'GCO2_1_MATH', 'GCO2_1_MATH_YEAR', 'GCO2_2', 'GCO2_2_MATH', 'GCO2_2_MATH_YEAR', 'GUS', 'GUS1', 'GUS2', 'G_RATE_AVG', 'G_RATE_YEAR', 'G_TOE_AVG', 'G_TOE_YEAR', 'HCO2_1', 'HCO2_2', 'HOSTCODE', 'HUS', 'HUS1', 'HUS2', 'H_RATE_AVG', 'H_RATE_YEAR', 'MM_TYPE', 'MON', 'NOW_CO2', 'REG_DATE', 'TOTAL_CO2_AVG', 'TOTAL_CO2_YEAR', 'TOTAL_RATE_AVG', 'TOTAL_RATE_YEAR', 'TYPE_DIV', 'TYPE_DIV_CD', 'WCO2_1', 'WCO2_1_MATH', 'WCO2_1_MATH_YEAR', 'WCO2_2', 'WCO2_2_MATH', 'WCO2_2_MATH_YEAR', 'WUS', 'WUS1', 'WUS2', 'W_RATE_AVG', 'W_RATE_YEAR', 'YEAR', 'YEAR_CO2', 'month', 'year']

=== MM_TYPE 유니크 값(상위 30개) ===


,count
MM_TYPE,
개인,42148


개인 행 수: 42148


,YEAR,MON,HOSTCODE,DONGCODE,MM_TYPE,TYPE_DIV,TYPE_DIV_CD,CNT,EUS,EUS1,...,NOW_CO2,YEAR_CO2,AVG_CO2,TOTAL_CO2_YEAR,TOTAL_CO2_AVG,TOTAL_RATE_YEAR,TOTAL_RATE_AVG,REG_DATE,year,month
0,2015,01,중랑구,면목3.8동,개인,아파트,TB_APT_C,1,215,250,...,91.16,106,106,-14.84,-14.84,-14,-14,2015-06-04 17:03:55.0,2015,1
1,2015,01,강동구,강일동,개인,아파트,TB_APT_Y,4969,1369580,1398337,...,678764.892,688765.44,693140.29,-10000.548,-14375.398,-1.452,-2.074,2015-06-04 17:03:55.0,2015,1
2,2015,01,강동구,강일동,개인,아파트외,TB_APT_N,14,3639,3014,...,3338.2952,2886.348,3339.912,451.9472,-1.6168,15.658,-0.048,2015-06-04 17:03:55.0,2015,1
3,2015,01,강동구,상일1동,개인,아파트,TB_APT_Y,2205,376695,377894,...,459449.096,446388.6,474691.028,13060.496,-15241.932,2.926,-3.211,2015-06-04 17:03:55.0,2015,1
4,2015,01,강동구,상일1동,개인,아파트외,TB_APT_N,1393,173914,172003,...,464982.06,444897.4824,484627.4204,20084.5776,-19645.3604,4.514,-4.054,2015-06-04 17:03:55.0,2015,1


In [21]:
# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# API 설정
api_key = '7657646572696e7931313561474d6b41'

# 데이터 수집
all_data = []

print("=== 데이터 수집 시작 ===")
for year in range(2015, 2025):
    for month in range(1, 13):
        # URL 생성
        url = f"http://openapi.seoul.go.kr:8088/{api_key}/json/energyUseDataStatInfo/1/1000/{year}/{month:02d}/"

        # API 호출
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()

            # 데이터 추출
            if 'energyUseDataStatInfo' in data:
                if 'row' in data['energyUseDataStatInfo']:
                    rows = data['energyUseDataStatInfo']['row']

                    # 개인 유형만 필터링
                    for row in rows:
                        if row['MM_TYPE'] == '개인':
                            row['year'] = year
                            row['month'] = month
                            all_data.append(row)

                    print(f"[{year}-{month:02d}] 성공")

print(f"\n총 수집 데이터: {len(all_data)}건")

=== 데이터 수집 시작 ===
[2015-01] 성공
[2015-02] 성공
[2015-03] 성공
[2015-04] 성공
[2015-05] 성공
[2015-06] 성공
[2015-07] 성공
[2015-08] 성공
[2015-09] 성공
[2015-10] 성공
[2015-11] 성공
[2015-12] 성공
[2016-01] 성공
[2016-02] 성공
[2016-03] 성공
[2016-04] 성공
[2016-05] 성공
[2016-06] 성공
[2016-07] 성공
[2016-08] 성공
[2016-09] 성공
[2016-10] 성공
[2016-11] 성공
[2016-12] 성공
[2017-01] 성공
[2017-02] 성공
[2017-03] 성공
[2017-04] 성공
[2017-05] 성공
[2017-06] 성공
[2017-07] 성공
[2017-08] 성공
[2017-09] 성공
[2017-10] 성공
[2017-11] 성공
[2017-12] 성공
[2018-01] 성공
[2018-02] 성공
[2018-03] 성공
[2018-04] 성공
[2018-05] 성공
[2018-06] 성공
[2018-07] 성공
[2018-08] 성공
[2018-09] 성공
[2018-10] 성공
[2018-11] 성공
[2018-12] 성공
[2019-01] 성공
[2019-02] 성공
[2019-03] 성공
[2019-04] 성공
[2019-05] 성공
[2019-06] 성공
[2019-07] 성공
[2019-08] 성공
[2019-09] 성공
[2019-10] 성공
[2019-11] 성공
[2019-12] 성공
[2020-01] 성공
[2020-02] 성공
[2020-03] 성공
[2020-04] 성공
[2020-05] 성공
[2020-06] 성공
[2020-07] 성공
[2020-08] 성공
[2020-09] 성공
[2020-10] 성공
[2020-11] 성공
[2020-12] 성공
[2021-01] 성공
[2021-02] 성공
[2021-03] 성공
[2021-0

**2-1: DataFrame 변환**

In [22]:
df = pd.DataFrame(all_data)

print("\n=== 문제 2-1: 데이터 기본 정보 ===")
print(f"총 데이터 수: {len(df)}건")
print(f"\n컬럼 수: {len(df.columns)}개")

df.info()

print("\n상위 5개 데이터:")
print(df.head())

# 에너지 사용량 데이터 확인
print("\n=== 개인의 현년 에너지 사용량 데이터 샘플 ===")
print("전기(EUS), 가스(GUS), 수도(WUS), 지역난방(HUS) 데이터:")
print(df[['year', 'month', 'DONGCODE', 'EUS', 'GUS', 'WUS', 'HUS']].head(10))


=== 문제 2-1: 데이터 기본 정보 ===
총 데이터 수: 42148건

컬럼 수: 62개
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42148 entries, 0 to 42147
Data columns (total 62 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   YEAR              42148 non-null  object
 1   MON               42148 non-null  object
 2   HOSTCODE          42148 non-null  object
 3   DONGCODE          42148 non-null  object
 4   MM_TYPE           42148 non-null  object
 5   TYPE_DIV          42148 non-null  object
 6   TYPE_DIV_CD       42148 non-null  object
 7   CNT               42148 non-null  object
 8   EUS               42148 non-null  object
 9   EUS1              42148 non-null  object
 10  EUS2              42148 non-null  object
 11  ECO2_1_MATH_YEAR  42148 non-null  object
 12  ECO2_2_MATH_YEAR  42148 non-null  object
 13  ECO2_1_MATH       42148 non-null  object
 14  ECO2_2_MATH       42148 non-null  object
 15  ECO2_1            42148 non-null  object
 16  ECO2

**2-2: 연도 및 계절 컬럼 추가**

In [10]:
def get_season(month):
    if month >= 3 and month <= 5:
        return '봄'
    elif month >= 6 and month <= 8:
        return '여름'
    elif month >= 9 and month <= 11:
        return '가을'
    else:
        return '겨울'

df['season'] = df['month'].apply(get_season)

print("\n=== 과제 2-2: 연도 및 계절 컬럼 추가 결과 ===")
print(df[['year', 'month', 'season']].head(15))

print("\n계절별 데이터 수:")
print(df['season'].value_counts())

=== year / month / season 확인 ===


,year,month,season
0,2015,1,겨울
7,2015,2,겨울
19,2015,3,봄
27,2015,4,봄
28,2015,5,봄


**3-1: 연도별 총 에너지 사용량 시각화**

In [11]:
# 숫자 변환
df['EUS'] = pd.to_numeric(df['EUS'], errors='coerce')
df['GUS'] = pd.to_numeric(df['GUS'], errors='coerce')
df['WUS'] = pd.to_numeric(df['WUS'], errors='coerce')
df['HUS'] = pd.to_numeric(df['HUS'], errors='coerce')

# 결측치 0으로 처리
df['EUS'] = df['EUS'].fillna(0)
df['GUS'] = df['GUS'].fillna(0)
df['WUS'] = df['WUS'].fillna(0)
df['HUS'] = df['HUS'].fillna(0)

# 총 에너지 사용량 계산
df['total_energy'] = df['EUS'] + df['GUS'] + df['WUS'] + df['HUS']

# 연도별 총합 계산
yearly_total = df.groupby('year')['total_energy'].sum()

print("\n=== 문제 3-1: 연도별 총 에너지 사용량 ===")
print(yearly_total)

# 그래프 그리기
plt.figure(figsize=(12, 6))
plt.plot(yearly_total.index, yearly_total.values, marker='o', linewidth=2)
plt.title('연도별 에너지 사용 총액 변화 - 3562', fontsize=14)
plt.xlabel('연도', fontsize=12)
plt.ylabel('총 에너지 사용량', fontsize=12)
plt.grid(True)
plt.savefig('연도별_에너지_사용량_3562.png', dpi=300)
plt.show()

연도별 합계(개인):


,EUS,GUS,WUS,HUS,total_usage
year,,,,,
2015,2307639393,3.300242e+08,1.623915e+08,9.976170e+07,2.899817e+09
2016,2624619777,3.692934e+08,1.639438e+08,1.020500e+08,3.259907e+09
2017,2725339685,4.089969e+08,1.629142e+08,1.120190e+08,3.409270e+09
2018,2943309193,4.591039e+08,1.633478e+08,1.164322e+08,3.682193e+09
2019,2983329866,4.430955e+08,1.688443e+08,1.954659e+08,3.790736e+09
2020,3147833086,4.417741e+08,1.775491e+08,1.390113e+08,3.906168e+09
2021,3357544310,4.445310e+08,1.799091e+08,1.231913e+08,4.105176e+09
2022,3353529606,4.628330e+08,1.762838e+08,1.399096e+08,4.132556e+09
2023,3400871711,4.153717e+08,1.740865e+08,1.304122e+08,4.120742e+09


계절별 가스 평균(개인):


,GUS
season,
가을,1.501570e+07
겨울,6.657663e+07
봄,4.443600e+07
여름,1.127243e+07
